# MPIN TASK

In [10]:
import datetime

COMMON_MPINS_FOUR_DIGIT = {
    "1234", "1111", "0000", "1212", "1122", "6969", "4321", "9999",
    "2222", "5555", "6666", "7777", "2000", "1004", "1313", "8888",
    "1230", "9876", "1010", "1100", "1331", "1441", "1001", "1221",
    "2112", "3113", "4004", "4114", "1224", "2412", "2020", "0101", "4567"
}

COMMON_MPINS_SIX_DIGIT = {
    "123456", "111111", "000000", "121212", "654321", "999999", "666666", "888888"
}

DATE_FORMAT_INPUT = "%d-%m-%Y"
DATE_PATTERNS_FOUR_DIGIT_INPUT= ["%d%m", "%m%d", "%y%m", "%m%y", "%y%d", "%d%y", "%Y", "%y"]
DATE_PATTERNS_SIX_DIGIT_INPUT = ["%d%m%y", "%m%d%y", "%y%m%d", "%d%m%Y", "%Y%m%d", "%Y", "%y"]

def extract_date_variant_parts(date_str, patterns, fmt=DATE_FORMAT):
    try:
        date_object= datetime.datetime.strptime(date_str, fmt)
        return {date.strftime(p) for p in patterns}
    except ValueError:
        return set()


In [22]:
def check_mpin_strengthen_security(pin, dob_self=None, dob_spouse=None, anniversary=None):
    vulnerability_reasons= []
    is_pin_weak = False

    pin_length = len(pin)
    common_pins = COMMON_MPINS_FOUR_DIGIT if pin_length == 4 else COMMON_MPINS_SIX_DIGIT
    date_patterns =DATE_PATTERNS_FOUR_DIGIT_INPUT if pin_length == 4 else DATE_PATTERNS_SIX_DIGIT_INPUT

    if pin in common_pins:
        is_pin_weak = True
        vulnerability_reasons.append("COMMONLY_USED")

    def match_demographic(date_str, reason_code):
        parts = extract_date_parts(date_str, date_patterns)
        if pin in parts:
            vulnerability_reasons.append(reason_code)
            return True
        return False

    if dob_self:
        is_pin_weak |= match_demographic(dob_self, "DEMOGRAPHIC_DOB_SELF")
    if dob_spouse:
        is_pin_weak |= match_demographic(dob_spouse, "DEMOGRAPHIC_DOB_SPOUSE")
    if anniversary:
        is_pin_weak |= match_demographic(anniversary, "DEMOGRAPHIC_ANNIVERSARY")

    return {
        "strength": "WEAK" if is_pin_weak else "STRONG",
        "reasons": vulnerability_reasons
    }

In [23]:
# -------------------- Test Cases --------------------
test_cases = [
    ("1234", None, None, None),
    ("0298", "02-01-1998", None, None),
    ("9802", "02-01-1998", None, None),
    ("0102", None, "01-01-2000", None),
    ("1202", None, None, "12-02-2022"),
    ("8888", None, None, None),
    ("4567", None, None, None),
    ("1998", "02-01-2004", None, None),
    ("1998", None, "02-01-1998", None),
    ("1998", None, None, "02-01-1998"),
    ("4321", None, None, None),
    ("2501", "25-01-2000", None, None),
    ("2512", None, None, "25-12-2020"),
    ("2020", None, None, "01-01-2020"),
    ("0102", "01-02-1995", None, None),
    ("123456", "12-03-2001", None, None),
    ("010203", "01-02-2003", None, None),
    ("000000", None, None, None),
    ("987654", None, None, None),
    ("250504", "25-05-2004", None, None)
]

for i, (pin, dob, spouse_dob, anniv) in enumerate(test_cases, 1):
    result = check_mpin_strengthen_security(pin, dob, spouse_dob, anniv)
    print(f"Test {i:02}: PIN = {pin} -> Strength: {result['strength']} | Reasons: {result['reasons']}")


Test 01: PIN = 1234 -> Strength: WEAK | Reasons: ['COMMONLY_USED']
Test 02: PIN = 0298 -> Strength: WEAK | Reasons: ['DEMOGRAPHIC_DOB_SELF']
Test 03: PIN = 9802 -> Strength: WEAK | Reasons: ['DEMOGRAPHIC_DOB_SELF']
Test 04: PIN = 0102 -> Strength: STRONG | Reasons: []
Test 05: PIN = 1202 -> Strength: WEAK | Reasons: ['DEMOGRAPHIC_ANNIVERSARY']
Test 06: PIN = 8888 -> Strength: WEAK | Reasons: ['COMMONLY_USED']
Test 07: PIN = 4567 -> Strength: WEAK | Reasons: ['COMMONLY_USED']
Test 08: PIN = 1998 -> Strength: STRONG | Reasons: []
Test 09: PIN = 1998 -> Strength: WEAK | Reasons: ['DEMOGRAPHIC_DOB_SPOUSE']
Test 10: PIN = 1998 -> Strength: WEAK | Reasons: ['DEMOGRAPHIC_ANNIVERSARY']
Test 11: PIN = 4321 -> Strength: WEAK | Reasons: ['COMMONLY_USED']
Test 12: PIN = 2501 -> Strength: WEAK | Reasons: ['DEMOGRAPHIC_DOB_SELF']
Test 13: PIN = 2512 -> Strength: WEAK | Reasons: ['DEMOGRAPHIC_ANNIVERSARY']
Test 14: PIN = 2020 -> Strength: WEAK | Reasons: ['COMMONLY_USED', 'DEMOGRAPHIC_ANNIVERSARY']
T